1 punto) Implementar la clase DatosClimaticos que reciba como parámetro la ruta
del archivo CSV temperaturas.csv que contiene datos climáticos. La clase debe
cargar los datos en un DataFrame de Pandas y procesarlos para añadir las
columnas 'Mes' y 'Año'.

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import folium

class DatosClimaticos:
    def __init__(self,ruta_archivo):
        #recibimos el archivo csv para generar el dataframe
        self.datos = pd.read_csv(ruta_archivo, sep=";", encoding='')
        
    def procesar_datos(self):
        # convertimos la columna en objeto de fecha para hacer calculos
        self.datos['Fecha'] = pd.to_datetime(self.datos['Fecha'])
        #usamos de fecha el mestemperaturas.csv
        self.datos['Mes'] = self.datos['Fecha'].dt.month
        #usamos de fecha el año
        self.datos['Año'] = self.datos['Fecha'].dt.year
        
    def obtener_datos(self):
        return self.datos

In [13]:
d_c= DatosClimaticos('temperaturas_nelson.csv')

#d_c.datos
d_c.obtener_datos()

d_c.procesar_datos()

In [14]:
import sqlite3
class AdminBaseDatos:
    def __init__(self, nombre_base_datos):
       self.nombre_base_datos = nombre_base_datos
       self.conexion = None #se usa por si el estado de conexion es NONE
 
    def conectar(self):
       self.conexion = sqlite3.connect(self.nombre_base_datos)
       print("Conexión establecida con la base de datos")
 
 
    def desconectar(self):
       self.conexion.close()
       print("Conexión cerrada con la base de datos")
        
       
    def crear_tablas_climatico(self):
        if self.conexion is None:
            print("No se ha establecido la conexión con la base de datos.")
            return
 
 
        cursor = self.conexion.cursor()
 
 
        # Crear tabla localizacion 
        cursor.execute('''CREATE TABLE IF NOT EXISTS localizacion (
                            Ciudad VARCHAR2(100) NOT NULL,
                            Latitud INTEGER NOT NULL,
                            Longitud INTEGER NOT NULL
                            
                        )''')
        print("Tabla 'localizacion' creada correctamente.")
 
 
        # Crear tabla temperatura
        cursor.execute('''CREATE TABLE IF NOT EXISTS temperatura (
                            Ciudad VARCHAR2(100) NOT NULL,
                            Fecha VARCHAR(100) NOT NULL,
                            Temperatura  INTEGER NOT NULL
                        )''')
        print("Tabla 'temperatura' creada correctamente.")
 
 
 
        self.conexion.commit()
       
    def insertar_datos(self, tabla, datos):
        if self.conexion is None:
            print("No se ha establecido la conexión con la base de datos.")
            return
       
       #convierte los datos proporcionados en una lista de tuplas 
       # utilizando el método itertuples() del objeto datos. 
        #listatuplas = list(datos.itertuples(index=False, name=None))
       
       #cursor para ejecutar comandos SQL.
        cursor = self.conexion.cursor()
 
 
        if tabla == "localizacion":
            cursor.executemany(f"INSERT INTO {tabla} VALUES (?, ?, ?)", datos)
        elif tabla == "temperatura":
            cursor.executemany(f"INSERT INTO {tabla} VALUES (?, ?, ?)", datos)
        else:
            print(f"No se puede insertar en la tabla '{tabla}'. Tabla no válida.")
            return
 
 
        self.conexion.commit()
        print(f"¡Datos insertados en la tabla '{tabla}' correctamente!")
 

In [15]:
admin_bd= AdminBaseDatos('/home/usuario/Escritorio/Programacion/Inicio_Librerias_BD_en_Python/Examen 3 trimestre/bd_temperatura_nelson.db')

#admin_bd.conectar()

#admin_bd.crear_tablas_climatico()

#admin_bd.insertar_datos('localizacion', [('sevilla', 10, 15)])

#admin_bd.desconectar()





In [16]:
d_c.datos

,Ciudad,Fecha,Temperatura,Latitud,Longitud,Mes,Año
0,Londres,2021-01-01,8,51.5074,-0.1278,1,2021
1,Londres,2021-01-02,7,51.5074,-0.1278,1,2021
2,Londres,2021-01-03,6,51.5074,-0.1278,1,2021
3,Londres,2021-01-04,6,51.5074,-0.1278,1,2021
4,Londres,2021-01-05,7,51.5074,-0.1278,1,2021
...,...,...,...,...,...,...,...
5835,Atenas,2022-12-27,10,37.9838,23.7275,12,2022
5836,Atenas,2022-12-28,11,37.9838,23.7275,12,2022
5837,Atenas,2022-12-29,14,37.9838,23.7275,12,2022
5838,Atenas,2022-12-30,14,37.9838,23.7275,12,2022


In [17]:
class AnalizadorClimatico:
    def __init__(self, datos_ventas):
        #accede al dataframe con sus atributos con (.datos)
        self.datos = datos_ventas.datos
          
    def obtener_temperatura_promedio_por_ciudad_y_mes(self):
        #del dataframe self.datos agrupa los nuevos campos (año, mes)
        # y pone la media (por grupos) ->Idx Año  Mes   Ventas
        #                                0  2021  Ene  1000.00
        return self.datos.groupby(['Ciudad', 'Mes'])['Temperatura'].mean(
        ).reset_index() #reset index: crea una secuencia numerica 
                        #con indices para usarla con funciones
        
    def obtener_temperatura_promedio_por_ciudad_y_año(self):
        return self.datos.groupby(['Ciudad', 'Año'])['Temperatura'].mean(
        ).reset_index() 
                        
                        
    def obtener_temperatura_promedio_por_ciudad(self):
        return self.datos.groupby(['Ciudad', 'Latitud', 'Longitud'])['Temperatura'].mean(
        ).reset_index() 
        
        
    def obtener_temperatura_maxima_por_ciudad(self):
        #agrupa por productos, suma los grupos de los productos y 
        # selecciona los productos que tienen mas ventas 
        return self.datos.groupby('Ciudad')['Temperatura'].max(
        ).reset_index()
        
    def obtener_temperatura_minima_por_ciudad(self):
        return self.datos.groupby('Ciudad')['Temperatura'].min(
        ).reset_index()
        
    def obtener_ciudades_mas_calurosas(self, n):
        return self.datos.groupby('Ciudad')['Temperatura'].sum(
        ).nlargest(n).reset_index()

In [18]:
a_v= AnalizadorClimatico(d_c)

a_v.obtener_ciudades_mas_calurosas(2)
a_v.obtener_temperatura_maxima_por_ciudad()
a_v.obtener_temperatura_minima_por_ciudad()
a_v.obtener_temperatura_promedio_por_ciudad()
a_v.obtener_temperatura_promedio_por_ciudad_y_año()
a_v.obtener_temperatura_promedio_por_ciudad_y_mes()

,Ciudad,Mes,Temperatura
0,Atenas,1,9.741935
1,Atenas,2,10.107143
2,Atenas,3,12.290323
3,Atenas,4,15.650000
4,Atenas,5,20.596774
...,...,...,...
91,Viena,8,19.032258
92,Viena,9,14.866667
93,Viena,10,10.177419
94,Viena,11,4.066667


In [19]:
class GeneradorResumenTemperaturas:
    def __init__(self, datos_temperatura, n):
       self.datos = datos_temperatura
       self.n= n #se usa para poner el parametro de la funcion ciudades_mas_calurosa 


    def generar_resumen_temperaturas(self):
        #de los dataframe que fueron procesado en analizador climatica
        #guardamos un resumen del histórico de temperaturas registradas
        tabla_1= self.datos.obtener_temperatura_promedio_por_ciudad_y_mes()

        table_2= self.datos.obtener_temperatura_promedio_por_ciudad_y_año()

        table_3= self.datos.obtener_temperatura_promedio_por_ciudad()

        table_4= self.datos.obtener_temperatura_maxima_por_ciudad()
        
        table_5= self.datos.obtener_temperatura_minima_por_ciudad()
        
        table_6= self.datos.obtener_ciudades_mas_calurosas(self.n)

        archivo_resumen_temperatura= 'resumen_temperaturas.txt'

        with open(archivo_resumen_temperatura, 'w') as resumen_temperatura:
            resumen_temperatura.write("Resumen histórico de temperatura registrada: ")
            resumen_temperatura.write("###############################################" +"\n")
            resumen_temperatura.write("\n")
            resumen_temperatura.write("Temperatura_promedio_por_ciudad_y_mes: ")
            resumen_temperatura.write("\n")
            resumen_temperatura.write(tabla_1.to_string(index= False))
            resumen_temperatura.write("\n")
            resumen_temperatura.write("###########################################" + "\n")
            resumen_temperatura.write("\n")
            resumen_temperatura.write("Promedio temperatura por ciudad y año \n")
            resumen_temperatura.write(table_2.to_string(index=False))
            resumen_temperatura.write("\n")
            resumen_temperatura.write("###########################################" + "\n")
            resumen_temperatura.write("\n")
            resumen_temperatura.write("Promedio temperatura por ciudad \n")
            resumen_temperatura.write(table_3.to_string(index=False))
            resumen_temperatura.write("\n")
            resumen_temperatura.write("###########################################" + "\n")
            resumen_temperatura.write("\n")
            resumen_temperatura.write("\n")
            resumen_temperatura.write("Temperatura maxima \n")
            resumen_temperatura.write(table_4.to_string(index=False))
            resumen_temperatura.write("\n")
            resumen_temperatura.write("###########################################" + "\n")
            resumen_temperatura.write("\n")
            resumen_temperatura.write("\n")
            resumen_temperatura.write("Temperatura minima\n")
            resumen_temperatura.write(table_5.to_string(index=False))
            resumen_temperatura.write("\n")
            resumen_temperatura.write("###########################################" + "\n")
            resumen_temperatura.write("\n")
            resumen_temperatura.write("\n")
            resumen_temperatura.write("Las 5 ciudades mas calurosas \n")
            resumen_temperatura.write(table_6.to_string(index=False))
            resumen_temperatura.write("\n")
            resumen_temperatura.write("###########################################" + "\n")
            resumen_temperatura.write("\n")

        print(f"¡Se ha generado el archivo de histórico de temperaturas registradas!")

In [20]:
fichero= GeneradorResumenTemperaturas(a_v, 2)

fichero.generar_resumen_temperaturas()

¡Se ha generado el archivo de histórico de temperaturas registradas!


In [21]:

d_c.obtener_datos()

,Ciudad,Fecha,Temperatura,Latitud,Longitud,Mes,Año
0,Londres,2021-01-01,8,51.5074,-0.1278,1,2021
1,Londres,2021-01-02,7,51.5074,-0.1278,1,2021
2,Londres,2021-01-03,6,51.5074,-0.1278,1,2021
3,Londres,2021-01-04,6,51.5074,-0.1278,1,2021
4,Londres,2021-01-05,7,51.5074,-0.1278,1,2021
...,...,...,...,...,...,...,...
5835,Atenas,2022-12-27,10,37.9838,23.7275,12,2022
5836,Atenas,2022-12-28,11,37.9838,23.7275,12,2022
5837,Atenas,2022-12-29,14,37.9838,23.7275,12,2022
5838,Atenas,2022-12-30,14,37.9838,23.7275,12,2022


In [22]:
class VisualizadorClimatico:
    def __init__(self, datos_temperatura):
        self.datos = datos_temperatura.obtener_datos()
        
    def graficar_temperatura_promedio_por_mes(self):
        #agrupa los campos año y mes y los une con la media de ventas 
        temperatura_promedio = self.datos.groupby(['Mes'])['Temperatura'].mean().reset_index()

        #ponemos en x mes y y en ventas
        plt.plot(temperatura_promedio['Mes'], temperatura_promedio['Temperatura'])
        
        plt.title('temperatura promedio por mes')
        
        plt.xlabel('Mes')
        
        plt.ylabel('Temperatura')
        
        plt.show()
        
    def graficar_ciudades_calurosas(self, n):
        ciudades_cal = self.datos.groupby('Mes')[
            'Temperatura'].max().nlargest(n).reset_index()
        
        #bar: gráfico de barras eje x y y
        plt.bar(ciudades_cal['Mes'], ciudades_cal[
            'Temperatura'])
        
        #muestra el parametro de n
        plt.title('Las {} ciudades mas calurosa'.format(n))
        plt.xlabel('Temperatura')
        plt.ylabel('Ciudad')
        plt.show()

    #def generar_mapa_climatico(self):
    #    m = folium.Map(location=[51.503364083692404, -0.11954297116416782],zoom_start=5)
    #    tooltip = "Ubicación"
#
    #    df = pd.read_csv('temperaturas_nelson.csv', sep=";")
#
    #    #print(df.columns)
#
    #    for i in range(len(df["Latitud"])):
    #        for j in range(len(df["Longitud"])):
    #            if i == j:
    #                folium.Marker([df["Latitud"][i], df["Longitud"][j]], 
    #                      tooltip=tooltip).add_to(m)
    #    
    #    return m
    

In [23]:
#recuerda poner en el parametro el objeto del contructor creado
g_t= VisualizadorClimatico(d_c)

#g_t.graficar_temperatura_promedio_por_mes()

#g_t.graficar_ciudades_calurosas(8)

#g_t.generar_mapa_climatico()

